In [28]:
import tarfile
import scipy.io
import pandas as pd
import numpy as np
import os
import shutil
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import datasets, layers, models

In [29]:
new_mat = scipy.io.loadmat('./Datasets/car/cars_devkit/cars_annos.mat')#car_annos.mat can be found on the Stanford car dataset
trial = np.hstack(new_mat['annotations'])
trial_df = pd.DataFrame(trial)
trial_df['file_name'] = trial_df['relative_im_path']
trial_df['file_name'] = trial_df['file_name'].map(np.array2string)
trial_df['file_name'] = trial_df['file_name'].map(lambda x:x.lstrip("['car_ims/").rstrip("']"))
trial_df['class_number'] = trial_df['class'].astype(float)
names= new_mat['class_names'].copy()
names_df = pd.DataFrame(names).T
names_df['class_number'] =names_df.index+1
names_df['car_name'] = names_df[0].map(lambda x:np.array2string(x).lstrip("['").rstrip("']"))
names_df = names_df.drop(columns = 0)
names_df['brand'] = names_df['car_name'].map(lambda x:x.split()[0])
names_df['model'] = names_df['car_name'].map(lambda x:x.split()[1])
names_df['both']= names_df['brand'] +  names_df['model']
names_df['both'] = names_df['both'].replace('RamC/V','RamCV')

In [30]:
names_df['both'] = names_df['both'].map(lambda x : x.lower())

In [31]:
merged_df_2 = trial_df.merge(names_df, left_on='class_number', right_on = 'class_number')
merged_df_2

,relative_im_path,bbox_x1,bbox_y1,bbox_x2,bbox_y2,class,test,file_name,class_number,car_name,brand,model,both
0,[car_ims/000001.jpg],[[112]],[[7]],[[853]],[[717]],[[1]],[[0]],000001.jpg,1.0,AM General Hummer SUV 2000,AM,General,amgeneral
1,[car_ims/000002.jpg],[[48]],[[24]],[[441]],[[202]],[[1]],[[0]],000002.jpg,1.0,AM General Hummer SUV 2000,AM,General,amgeneral
2,[car_ims/000003.jpg],[[7]],[[4]],[[277]],[[180]],[[1]],[[0]],000003.jpg,1.0,AM General Hummer SUV 2000,AM,General,amgeneral
3,[car_ims/000004.jpg],[[33]],[[50]],[[197]],[[150]],[[1]],[[0]],000004.jpg,1.0,AM General Hummer SUV 2000,AM,General,amgeneral
4,[car_ims/000005.jpg],[[5]],[[8]],[[83]],[[58]],[[1]],[[0]],000005.jpg,1.0,AM General Hummer SUV 2000,AM,General,amgeneral
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16180,[car_ims/016181.jpg],[[38]],[[36]],[[375]],[[234]],[[196]],[[1]],016181.jpg,196.0,smart fortwo Convertible 2012,smart,fortwo,smartfortwo
16181,[car_ims/016182.jpg],[[29]],[[34]],[[235]],[[164]],[[196]],[[1]],016182.jpg,196.0,smart fortwo Convertible 2012,smart,fortwo,smartfortwo
16182,[car_ims/016183.jpg],[[25]],[[32]],[[587]],[[359]],[[196]],[[1]],016183.jpg,196.0,smart fortwo Convertible 2012,smart,fortwo,smartfortwo
16183,[car_ims/016184.jpg],[[56]],[[60]],[[208]],[[186]],[[196]],[[1]],016184.jpg,196.0,smart fortwo Convertible 2012,smart,fortwo,smartfortwo


In [32]:
from sklearn.model_selection import train_test_split


In [33]:
train, test = train_test_split(merged_df_2, test_size = 0.2, random_state = 42)


In [34]:
train

,relative_im_path,bbox_x1,bbox_y1,bbox_x2,bbox_y2,class,test,file_name,class_number,car_name,brand,model,both
15775,[car_ims/015776.jpg],[[296]],[[398]],[[1588]],[[995]],[[192]],[[0]],015776.jpg,192.0,Volkswagen Beetle Hatchback 2012,Volkswagen,Beetle,volkswagenbeetle
6759,[car_ims/006760.jpg],[[49]],[[145]],[[532]],[[405]],[[83]],[[1]],006760.jpg,83.0,Dodge Caliber Wagon 2012,Dodge,Caliber,dodgecaliber
8875,[car_ims/008876.jpg],[[17]],[[11]],[[240]],[[160]],[[109]],[[0]],008876.jpg,109.0,Ford Expedition EL SUV 2009,Ford,Expedition,fordexpedition
8919,[car_ims/008920.jpg],[[41]],[[32]],[[773]],[[489]],[[109]],[[1]],008920.jpg,109.0,Ford Expedition EL SUV 2009,Ford,Expedition,fordexpedition
10307,[car_ims/010308.jpg],[[30]],[[65]],[[545]],[[506]],[[125]],[[0]],010308.jpg,125.0,HUMMER H2 SUT Crew Cab 2009,HUMMER,H2,hummerh2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13418,[car_ims/013419.jpg],[[166]],[[597]],[[1949]],[[1431]],[[164]],[[0]],013419.jpg,164.0,Mercedes-Benz E-Class Sedan 2012,Mercedes-Benz,E-Class,mercedes-benze-class
5390,[car_ims/005391.jpg],[[27]],[[45]],[[628]],[[441]],[[67]],[[0]],005391.jpg,67.0,Chevrolet Malibu Hybrid Sedan 2010,Chevrolet,Malibu,chevroletmalibu
860,[car_ims/000861.jpg],[[43]],[[49]],[[570]],[[358]],[[11]],[[1]],000861.jpg,11.0,Aston Martin Virage Coupe 2012,Aston,Martin,astonmartin
15795,[car_ims/015796.jpg],[[195]],[[248]],[[854]],[[557]],[[192]],[[0]],015796.jpg,192.0,Volkswagen Beetle Hatchback 2012,Volkswagen,Beetle,volkswagenbeetle


In [35]:
carBrand = names_df['both'].unique()


In [36]:
for b in carBrand:#this loop is used to take a particular image from the car_ims folder, make a copy and put it in the 'train' folder of the particular car it belongs to. Make sure to the unzipped car_ims.tgz file from the Stanford website.
  temp_df = train[train['both']==b]
  temp_df.reset_index(drop = True)
  print(b)
  for i in list(temp_df.index.values):
    shutil.copy('./Datasets/car/car_ims/'+temp_df['file_name'][i], './Datasets/car/train_brand_and_model_2/'+b.lower()+'/'+temp_df['file_name'][i])
     

amgeneral
acurarl
acuratl
acuratsx
acuraintegra
acurazdx
astonmartin
audirs
audia5
auditts
audir8
audiv8
audi100
auditt
audis6
audis5
audis4
bmwactivehybrid
bmw1
bmw3
bmw6
bmwx5
bmwx6
bmwm3
bmwm5
bmwm6
bmwx3
bmwz4
bentleycontinental
bentleyarnage
bentleymulsanne
bugattiveyron
buickregal
buickrainier
buickverano
buickenclave
cadillaccts-v
cadillacsrx
cadillacescalade
chevroletsilverado
chevroletcorvette
chevrolettraverse
chevroletcamaro
chevrolethhr
chevroletimpala
chevrolettahoe
chevroletsonic
chevroletexpress
chevroletavalanche
chevroletcobalt
chevroletmalibu
chevrolettrailblazer
chevroletmonte
chrysleraspen
chryslersebring
chryslertown
chrysler300
chryslercrossfire
chryslerpt
daewoonubira
dodgecaliber
dodgecaravan
dodgeram
dodgesprinter
dodgejourney
dodgedakota
dodgemagnum
dodgechallenger
dodgedurango
dodgecharger
eagletalon
fiat500
ferrariff
ferraricalifornia
ferrari458
fiskerkarma
fordf-450
fordmustang
fordfreestar
fordexpedition
fordedge
fordranger
fordgt
fordf-150
fordfocus
forde

In [37]:
test = test.reset_index(drop = True)
test#this is the dataframe for the test set

,relative_im_path,bbox_x1,bbox_y1,bbox_x2,bbox_y2,class,test,file_name,class_number,car_name,brand,model,both
0,[car_ims/015759.jpg],[[46]],[[64]],[[527]],[[386]],[[191]],[[1]],015759.jpg,191.0,Volkswagen Golf Hatchback 1991,Volkswagen,Golf,volkswagengolf
1,[car_ims/014970.jpg],[[95]],[[140]],[[715]],[[419]],[[182]],[[1]],014970.jpg,182.0,Suzuki Kizashi Sedan 2012,Suzuki,Kizashi,suzukikizashi
2,[car_ims/006875.jpg],[[154]],[[277]],[[1453]],[[979]],[[85]],[[0]],006875.jpg,85.0,Dodge Caravan Minivan 1997,Dodge,Caravan,dodgecaravan
3,[car_ims/009759.jpg],[[19]],[[52]],[[286]],[[210]],[[119]],[[1]],009759.jpg,119.0,GMC Savana Van 2012,GMC,Savana,gmcsavana
4,[car_ims/013072.jpg],[[86]],[[237]],[[979]],[[588]],[[160]],[[0]],013072.jpg,160.0,McLaren MP4-12C Coupe 2012,McLaren,MP4-12C,mclarenmp4-12c
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3232,[car_ims/006696.jpg],[[48]],[[57]],[[518]],[[361]],[[82]],[[1]],006696.jpg,82.0,Daewoo Nubira Wagon 2002,Daewoo,Nubira,daewoonubira
3233,[car_ims/009437.jpg],[[29]],[[12]],[[623]],[[386]],[[115]],[[1]],009437.jpg,115.0,Ford Focus Sedan 2007,Ford,Focus,fordfocus
3234,[car_ims/009884.jpg],[[52]],[[36]],[[247]],[[189]],[[121]],[[0]],009884.jpg,121.0,GMC Acadia SUV 2012,GMC,Acadia,gmcacadia
3235,[car_ims/000511.jpg],[[95]],[[62]],[[402]],[[306]],[[7]],[[0]],000511.jpg,7.0,Acura ZDX Hatchback 2012,Acura,ZDX,acurazdx


In [38]:
for b in carBrand:#this loop is used to take a particular image from the car_ims folder, make a copy and put it in the 'test' folder of the particular car it belongs to.
  temp_df = test[test['both']==b]
  temp_df.reset_index(drop = True)
  print(b)
  for i in list(temp_df.index.values):
    shutil.copy('./Datasets/car/car_ims/'+temp_df['file_name'][i], './Datasets/car/test_brand_and_model_2/'+b.lower()+'/'+temp_df['file_name'][i])

amgeneral
acurarl
acuratl
acuratsx
acuraintegra
acurazdx
astonmartin
audirs
audia5
auditts
audir8
audiv8
audi100
auditt
audis6
audis5
audis4
bmwactivehybrid
bmw1
bmw3
bmw6
bmwx5
bmwx6
bmwm3
bmwm5
bmwm6
bmwx3
bmwz4
bentleycontinental
bentleyarnage
bentleymulsanne
bugattiveyron
buickregal
buickrainier
buickverano
buickenclave
cadillaccts-v
cadillacsrx
cadillacescalade
chevroletsilverado
chevroletcorvette
chevrolettraverse
chevroletcamaro
chevrolethhr
chevroletimpala
chevrolettahoe
chevroletsonic
chevroletexpress
chevroletavalanche
chevroletcobalt
chevroletmalibu
chevrolettrailblazer
chevroletmonte
chrysleraspen
chryslersebring
chryslertown
chrysler300
chryslercrossfire
chryslerpt
daewoonubira
dodgecaliber
dodgecaravan
dodgeram
dodgesprinter
dodgejourney
dodgedakota
dodgemagnum
dodgechallenger
dodgedurango
dodgecharger
eagletalon
fiat500
ferrariff
ferraricalifornia
ferrari458
fiskerkarma
fordf-450
fordmustang
fordfreestar
fordexpedition
fordedge
fordranger
fordgt
fordf-150
fordfocus
forde

In [1]:
import os
import shutil

source_dir = r'C:\MY FILES\CarDetective\Datasets\car\train_brand_and_model_2'
dest_dir = r'C:\MY FILES\CarDetective\Datasets\car\OwnCollection\vehicles'

# Check if the source directory exists
if not os.path.isdir(source_dir):
    print(f"Error: {source_dir} is not a directory.")
    exit()

# Check if the destination directory exists, if not create it
if not os.path.isdir(dest_dir):
    os.makedirs(dest_dir)

# Loop through all subdirectories in the source directory
for subdir, _, files in os.walk(source_dir):
    # Loop through all files in the subdirectory
    for file in files:
        # Check if the file is an image file
        if file.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp')):
            # Construct the source and destination paths
            src_path = os.path.join(subdir, file)
            dest_path = os.path.join(dest_dir, file)
            # Copy the file to the destination directory
            shutil.copy2(src_path, dest_path)
            print(f"Copied {src_path} to {dest_path}")


Copied C:\MY FILES\CarDetective\Datasets\car\train_brand_and_model_2\acuraintegra\000405.jpg to C:\MY FILES\CarDetective\Datasets\car\OwnCollection\vehicles\000405.jpg
Copied C:\MY FILES\CarDetective\Datasets\car\train_brand_and_model_2\acuraintegra\000406.jpg to C:\MY FILES\CarDetective\Datasets\car\OwnCollection\vehicles\000406.jpg
Copied C:\MY FILES\CarDetective\Datasets\car\train_brand_and_model_2\acuraintegra\000407.jpg to C:\MY FILES\CarDetective\Datasets\car\OwnCollection\vehicles\000407.jpg
Copied C:\MY FILES\CarDetective\Datasets\car\train_brand_and_model_2\acuraintegra\000413.jpg to C:\MY FILES\CarDetective\Datasets\car\OwnCollection\vehicles\000413.jpg
Copied C:\MY FILES\CarDetective\Datasets\car\train_brand_and_model_2\acuraintegra\000415.jpg to C:\MY FILES\CarDetective\Datasets\car\OwnCollection\vehicles\000415.jpg
Copied C:\MY FILES\CarDetective\Datasets\car\train_brand_and_model_2\acuraintegra\000416.jpg to C:\MY FILES\CarDetective\Datasets\car\OwnCollection\vehicles\000